In [ ]:
import pandas as pd
from pathlib import Path

current_dir = Path.cwd()

df = pd.read_csv(current_dir.parent/'house_data'/'extracted_houses_paginated.csv')


,link,strikethrough_price,sold_price,property_type,address,bedrooms,bathrooms,garages
0,/on/richmond-hill-real-estate/204-330-red-mapl...,"$600,000","$605,000",Condo Apt,"204 - 330 Red Maple Rd , Richmond Hill - Langs...",NaN,NaN,NaN
1,/on/etobicoke-real-estate/508-40-richview-rd/h...,"$729,888","$680,000",Condo Apt,"508 - 40 Richview Rd , Etobicoke - Humber Heights",NaN,NaN,NaN
2,/on/mississauga-real-estate/1616-50-mississaug...,"$619,900","$595,000",Condo Apt,"1616 - 50 Mississauga Valley Blvd , Mississaug...",NaN,NaN,NaN
3,/on/etobicoke-real-estate/24-wallis-cres/home/...,"$874,999","$815,000",Detached,"24 Wallis Cres , Etobicoke - Mount Olive-Silve...",NaN,NaN,NaN
4,/on/vaughan-real-estate/27-novella-rd/home/b1D...,"$1,198,000","$1,148,000",Semi-Detached,"27 Novella Rd , Vaughan - Patterson",NaN,NaN,NaN
5,/on/toronto-real-estate/ph03-159-wellesley-st-...,"$549,990","$525,000",Condo Apt,"PH03 - 159 Wellesley St E , Toronto - Cabbaget...",NaN,NaN,NaN
6,/on/markham-real-estate/25-ryler-way/home/9w8o...,"$1,256,900","$1,230,000",Semi-Detached,"25 Ryler Way , Markham - Cedarwood",NaN,NaN,NaN
7,/on/toronto-real-estate/2611-8-eglinton-ave-e/...,"$659,800","$650,000",Condo Apt,"2611 - 8 Eglinton Ave E , Toronto - Mount Plea...",NaN,NaN,NaN
8,/on/markham-real-estate/34-washington-st/home/...,"$1,429,000","$1,429,000",Detached,"34 Washington St , Markham - Old Markham Village",NaN,NaN,NaN
9,/on/etobicoke-real-estate/405-35-fontenay-crt/...,"$590,888","$588,000",Condo Apt,"405 - 35 Fontenay Crt , Etobicoke - Edenbridge...",NaN,NaN,NaN


In [ ]:
import pandas as pd
from tqdm import tqdm
import requests

tqdm.pandas()  # Enable progress bar for Pandas apply

def get_coordinates(address, api_key):
    """
    Fetch latitude and longitude for a given address using Google Geocoding API.
    
    Parameters:
        address (str): The address to geocode.
        api_key (str): API key for Google Geocoding API.
    
    Returns:
        tuple: Latitude and Longitude as (lat, long). Returns (None, None) if an error occurs.
    """
    try:
        # Construct the Geocoding API URL
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"
        params = {
            "address": address,
            "key": api_key
        }

        # Send the request to the API
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Parse the JSON response
        data = response.json()

        if data["status"] == "OK":
            # Extract latitude and longitude from the response
            location = data["results"][0]["geometry"]["location"]
            return location["lat"], location["lng"]
        else:
            print(f"Geocoding API error for address '{address}': {data['status']}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Request error for address '{address}': {e}")
        return None, None

def process_dataframe_in_chunks(df, chunk_size, api_key, output_csv):
    """
    Processes a DataFrame in chunks to find latitude and longitude for each row.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame containing the 'address' column.
        chunk_size (int): Number of rows to process in each chunk.
        api_key (str): API key for the geocoding service.
        output_csv (str): Filepath to append the results.

    Returns:
        None
    """
    # Check if the output CSV already exists
    try:
        processed_rows = pd.read_csv(output_csv).shape[0]
        print(f"Resuming from row {processed_rows}.")
    except FileNotFoundError:
        # Initialize the output file if it doesn't exist
        processed_rows = 0
        pd.DataFrame(columns=['address', 'lat', 'long']).to_csv(output_csv, index=False)

    # Process the DataFrame in chunks
    for start_idx in range(processed_rows, len(df), chunk_size):
        chunk = df.iloc[start_idx:start_idx + chunk_size].copy()  # Copy the chunk

        # Process each row in the chunk to get lat and long
        chunk[['lat', 'long']] = chunk['address'].progress_apply(
            lambda addr: pd.Series(get_coordinates(addr, api_key))
        )

        # Append the results to the CSV
        chunk.to_csv(output_csv, mode='a', header=False, index=False)
        print(f"Processed rows {start_idx} to {start_idx + len(chunk)} and saved to {output_csv}.")



# Filepath for the output CSV
output_csv = "geocoded_addresses.csv"

# API key for the geocoding service


# Process the DataFrame in chunks of 20 rows
process_dataframe_in_chunks(df, chunk_size=20, api_key=api_key, output_csv=output_csv)


100%|██████████| 20/20 [00:04<00:00,  4.32it/s]


Processed rows 0 to 20 and saved to geocoded_addresses.csv.


100%|██████████| 20/20 [00:02<00:00,  8.02it/s]


Processed rows 20 to 40 and saved to geocoded_addresses.csv.


100%|██████████| 20/20 [00:04<00:00,  4.64it/s]


...

Processed rows 3940 to 3960 and saved to geocoded_addresses.csv.


100%|██████████| 20/20 [00:05<00:00,  3.75it/s]


Processed rows 3960 to 3980 and saved to geocoded_addresses.csv.


100%|██████████| 20/20 [00:04<00:00,  4.22it/s]

Processed rows 3980 to 4000 and saved to geocoded_addresses.csv.
